In [1]:
# Inception model https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), # Specify our input shape
                                include_top = False, # We don't want fully-connected-layers, we want straight for convolution
                                weights = None) # We don't want to use the built-in weights, but the snapshot you've just downloaded (in link)

pre_trained_model.load_weights(local_weights_file)

In [ ]:
# We want to iterate each layer and lock them
for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

In [ ]:
# All of the layers have name. We can look up the name of the last layer we want to use.
# Using Summary, you'll see the layers convolved to 3x3. 
# But, we want to use something with mor einformation, so move up the model description to find mixed7.

last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [ ]:
# Now we define our new model. 

from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(
    optimizer = RMSprop(learning_rate=0.0001),
    loss = 'binary_crossentropy',
    metrics = ['acc']
)

# Then do the rest
#  model.fit blablabla

# After we do that, we get a different result of overfitting.
# The validation is diverging down in a bad way.